In [142]:
import pandas as pd

In [143]:
my_prediction = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\all_inbar_predictions.csv")
my_prediction

,position,pathogenicity,predictions,variant,gene
0,6,0,0,G6S,SLC26A4
1,21,0,1,M21V,SLC26A4
2,51,0,0,A51T,SLC26A4
3,53,0,1,C53G,SLC26A4
4,163,0,1,V163I,SLC26A4
...,...,...,...,...,...
4516,1045,1,1,G1045V,COL4A3
4517,871,1,1,G871C,COL4A3
4518,853,1,1,G853R,COL4A3
4519,25,0,0,P25S,COL4A3


In [144]:
tools = ["VEST"]

In [145]:
merged = my_prediction
for tool in tools:
    tool_predictions = pd.read_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\{tool}_on_dvd_data_predictions.csv")
    # Merge the two files
    merged = pd.merge(merged, tool_predictions, on=["gene", "variant"])
merged

,position,pathogenicity,predictions,variant,gene,Unnamed: 0,VEST_score,VEST_pathogenicity
0,6,0,0,G6S,SLC26A4,0,0.104,0
1,21,0,1,M21V,SLC26A4,1,0.037,0
2,51,0,0,A51T,SLC26A4,2,0.188,0
3,53,0,1,C53G,SLC26A4,3,0.062,0
4,180,0,1,A180T,SLC26A4,4,0.177,0
...,...,...,...,...,...,...,...,...
3548,1045,1,1,G1045V,COL4A3,3488,0.986,1
3549,871,1,1,G871C,COL4A3,3489,0.968,1
3550,853,1,1,G853R,COL4A3,3490,0.979,1
3551,25,0,0,P25S,COL4A3,3491,0.101,0


In [146]:
# Print how many variants there are from each gene
merged["gene"].value_counts()

### Notice that VEST4 doesn't have data of WFS1, and REVEL doesn't have data for SLC26A4.

MYO7A      1097
COL2A1      839
COL4A5      597
COL4A3      358
FGFR1       354
SLC26A4     308
Name: gene, dtype: int64

In [147]:
## Calculate the MCC for the predictions, according to the real pathogenicity (0 or 1)
from sklearn.metrics import matthews_corrcoef

In [148]:
# separate the merged dataframe according to gene
genes = merged["gene"].unique()
print(f"Number of genes: {len(genes)}")

Number of genes: 6


In [149]:
# Calculate MCC for each gene specific predictor for mutpred

# Build dictionary with gene names as keys.
mccs = {gene: 0 for gene in genes}

for tool in tools:
    for gene in genes:
        gene_df = merged[merged["gene"] == gene]
        # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
        # Create binary predictions based on the 0.5 threshold
        
        gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
        
        # Calculate MCC
        mcc = matthews_corrcoef(gene_df['pathogenicity'], gene_df[f"{tool}_pathogenicity"])
        
        # Get gene name for the use for the dictionary
        gene = gene_df['gene'].unique()[0]
        
        # Append mcc to dictionary
        mccs[gene] = mcc
    
    print(f"MCCs of {tool} predictions for each gene:")
    for gene in mccs:
        print(f"{gene}: {mccs[gene]}")
        
# get MCC for my predictions
mccs = {gene: 0 for gene in genes}

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
    # Create binary predictions based on the 0.5 threshold
    
    gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
    gene_df['predictions'] = gene_df['predictions'].astype(int)
    
    # Calculate MCC
    mcc = matthews_corrcoef(gene_df['pathogenicity'], gene_df['predictions'])
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    
    # Append mcc to dictionary
    mccs[gene] = mcc

print(f"MCCs of predictions for each gene:")
for gene in mccs:
    print(f"{gene}: {mccs[gene]}")


MCCs of VEST predictions for each gene:
SLC26A4: 0.907534543157961
FGFR1: 0.7315749764637576
COL2A1: 0.8806798282981236
COL4A5: 0.9584476056701803
MYO7A: 0.5677893215850368
COL4A3: 0.7942013386868483
MCCs of predictions for each gene:
SLC26A4: 0.09773691446699974
FGFR1: 0.5274720860822589
COL2A1: 0.6507754732637137
COL4A5: 0.33660891604674725
MYO7A: 0.4043404609098987
COL4A3: 0.5792191706990236


C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_23524\41423480.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_23524\41423480.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_23524\41423480.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [150]:
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, f1_score, log_loss, confusion_matrix
# store the results in a dictionary
results = {gene: 0 for gene in genes}

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    y_true = gene_df["pathogenicity"].astype(int)
    predicted_probabilities = gene_df["MutPred_score"]
    gene_df.loc[:, 'binary_prediction_mutpred'] = (gene_df['MutPred_score'] > 0.5).astype(int)
    predicted_labels = gene_df["binary_prediction_mutpred"]
    
    results_gene = {}
    # Calculate AUC-ROC
    roc_auc = roc_auc_score(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["roc_auc"] = roc_auc
    
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, predicted_probabilities)
    roc_auc = auc(fpr, tpr)
    # append to dictionary
    results_gene["auc"] = roc_auc
    
    # Calculate AUC-PR
    precision, recall, _ = precision_recall_curve(y_true, predicted_probabilities)
    pr_auc = auc(recall, precision)
    # append to dictionary
    results_gene["pr_auc"] = pr_auc
    
    # Calculate F1 Score
    f1 = f1_score(y_true, predicted_labels)
    # append to dictionary
    results_gene["f1"] = f1
    
    # Calculate Log Loss
    logloss = log_loss(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["logloss"] = logloss
    
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(y_true, predicted_labels)
    # append to dictionary
    results_gene["confusion_matrix"] = conf_matrix
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    # Append results to dictionary
    results[gene] = results_gene
    
## Print results
print("Results of MutPred2 predictions for each gene:")
for gene in results:
    print(f"{gene}: {results[gene]}")

KeyError: 'MutPred_score'

In [ ]:
# present the confusion matrix for each gene
import matplotlib.pyplot as plt

for gene in results:
    # Plot confusion matrix
    plt.imshow(results[gene]["confusion_matrix"], cmap=plt.cm.Blues)
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.xticks([0, 1], ["Benign", "Pathogenic"])
    plt.yticks([0, 1], ["Benign", "Pathogenic"])
    plt.title(f"Confusion matrix for {gene}, MutPred2 predictions")
    plt.style.use("seaborn-white")
    # Add text annotations
    for i in range(2):
        for j in range(2):
            plt.text(j, i, results[gene]["confusion_matrix"][i, j], ha="center", va="center", color="black", backgroundcolor="white")
    plt.colorbar()
    plt.show()

In [ ]:
# present the ROC curve for each gene
for gene in results:
    # Plot ROC curve
    plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {results[gene]['roc_auc']:.2f})")
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC curve for {gene}, MutPred2 predictions")
    plt.legend(loc="lower right")
    plt.show()